In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
from keras import regularizers
#tf.enable_eager_execution()

Using TensorFlow backend.


In [2]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [3]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image,max_delta=0.5)
    image = tf.image.random_contrast(image,lower=0.2,upper=2.0)
    image = tf.image.random_hue(image,max_delta=0.08)
    image = tf.image.random_saturation(image,lower=0.2,upper=2.0)
    return image, label

train_data = train_data.map(scale).shuffle(100000).repeat(3).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(256, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(512, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),   
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [5]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    '/content/gdrive/My Drive/checkpoints/best.hdf5',
    monitor='val_acc',
    save_best_only=True)

In [6]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [7]:
history = model.fit(train_data,
                   epochs=50,
                   validation_data=valid_data,
                   callbacks=[cp_callback])

Epoch 1/50


    168/Unknown - 14s 14s/step - loss: 4.0962 - accuracy: 0.09 - 14s 7s/step - loss: 4.4048 - accuracy: 0.0859 - 14s 5s/step - loss: 4.4032 - accuracy: 0.083 - 14s 3s/step - loss: 4.5090 - accuracy: 0.078 - 14s 3s/step - loss: 4.5060 - accuracy: 0.075 - 14s 2s/step - loss: 4.5177 - accuracy: 0.075 - 14s 2s/step - loss: 4.4994 - accuracy: 0.084 - 14s 2s/step - loss: 4.4876 - accuracy: 0.091 - 14s 2s/step - loss: 4.4354 - accuracy: 0.099 - 14s 1s/step - loss: 4.3948 - accuracy: 0.104 - 14s 1s/step - loss: 4.4210 - accuracy: 0.098 - 14s 1s/step - loss: 4.3870 - accuracy: 0.105 - 14s 1s/step - loss: 4.3628 - accuracy: 0.108 - 14s 1s/step - loss: 4.3644 - accuracy: 0.109 - 14s 937ms/step - loss: 4.3666 - accuracy: 0.10 - 14s 881ms/step - loss: 4.3463 - accuracy: 0.11 - 14s 831ms/step - loss: 4.3477 - accuracy: 0.11 - 14s 787ms/step - loss: 4.3226 - accuracy: 0.11 - 14s 748ms/step - loss: 4.3266 - accuracy: 0.11 - 14s 712ms/step - loss: 4.3060 - accuracy: 0.11 - 14s 680ms/step - loss: 4.2823

    335/Unknown - 20s 117ms/step - loss: 3.7157 - accuracy: 0.17 - 20s 117ms/step - loss: 3.7138 - accuracy: 0.17 - 20s 116ms/step - loss: 3.7130 - accuracy: 0.17 - 20s 116ms/step - loss: 3.7102 - accuracy: 0.17 - 20s 115ms/step - loss: 3.7082 - accuracy: 0.17 - 20s 115ms/step - loss: 3.7049 - accuracy: 0.17 - 20s 114ms/step - loss: 3.7058 - accuracy: 0.17 - 20s 114ms/step - loss: 3.7042 - accuracy: 0.17 - 20s 113ms/step - loss: 3.7033 - accuracy: 0.17 - 20s 113ms/step - loss: 3.7021 - accuracy: 0.17 - 20s 113ms/step - loss: 3.6993 - accuracy: 0.17 - 20s 112ms/step - loss: 3.6975 - accuracy: 0.17 - 20s 112ms/step - loss: 3.6944 - accuracy: 0.18 - 20s 111ms/step - loss: 3.6922 - accuracy: 0.18 - 20s 111ms/step - loss: 3.6933 - accuracy: 0.17 - 20s 110ms/step - loss: 3.6903 - accuracy: 0.17 - 20s 110ms/step - loss: 3.6888 - accuracy: 0.18 - 20s 110ms/step - loss: 3.6866 - accuracy: 0.18 - 20s 109ms/step - loss: 3.6832 - accuracy: 0.18 - 20s 109ms/step - loss: 3.6814 - accuracy: 0.18 - 21

    502/Unknown - 26s 77ms/step - loss: 3.4862 - accuracy: 0.211 - 26s 77ms/step - loss: 3.4845 - accuracy: 0.212 - 26s 77ms/step - loss: 3.4834 - accuracy: 0.212 - 26s 77ms/step - loss: 3.4823 - accuracy: 0.212 - 26s 77ms/step - loss: 3.4812 - accuracy: 0.212 - 26s 77ms/step - loss: 3.4808 - accuracy: 0.212 - 26s 77ms/step - loss: 3.4788 - accuracy: 0.212 - 26s 76ms/step - loss: 3.4774 - accuracy: 0.213 - 26s 76ms/step - loss: 3.4758 - accuracy: 0.213 - 26s 76ms/step - loss: 3.4742 - accuracy: 0.213 - 26s 76ms/step - loss: 3.4726 - accuracy: 0.213 - 26s 76ms/step - loss: 3.4713 - accuracy: 0.214 - 26s 76ms/step - loss: 3.4700 - accuracy: 0.214 - 26s 76ms/step - loss: 3.4680 - accuracy: 0.215 - 26s 76ms/step - loss: 3.4669 - accuracy: 0.215 - 27s 76ms/step - loss: 3.4655 - accuracy: 0.215 - 27s 75ms/step - loss: 3.4651 - accuracy: 0.215 - 27s 75ms/step - loss: 3.4644 - accuracy: 0.215 - 27s 75ms/step - loss: 3.4633 - accuracy: 0.215 - 27s 75ms/step - loss: 3.4618 - accuracy: 0.216 - 27

    669/Unknown - 32s 64ms/step - loss: 3.2927 - accuracy: 0.248 - 32s 64ms/step - loss: 3.2916 - accuracy: 0.249 - 32s 64ms/step - loss: 3.2905 - accuracy: 0.249 - 32s 64ms/step - loss: 3.2892 - accuracy: 0.249 - 32s 64ms/step - loss: 3.2884 - accuracy: 0.249 - 32s 64ms/step - loss: 3.2876 - accuracy: 0.250 - 32s 64ms/step - loss: 3.2871 - accuracy: 0.250 - 32s 63ms/step - loss: 3.2861 - accuracy: 0.250 - 32s 63ms/step - loss: 3.2850 - accuracy: 0.250 - 32s 63ms/step - loss: 3.2838 - accuracy: 0.250 - 32s 63ms/step - loss: 3.2831 - accuracy: 0.251 - 33s 63ms/step - loss: 3.2819 - accuracy: 0.251 - 33s 63ms/step - loss: 3.2809 - accuracy: 0.251 - 33s 63ms/step - loss: 3.2795 - accuracy: 0.251 - 33s 63ms/step - loss: 3.2783 - accuracy: 0.252 - 33s 63ms/step - loss: 3.2766 - accuracy: 0.252 - 33s 63ms/step - loss: 3.2756 - accuracy: 0.252 - 33s 63ms/step - loss: 3.2746 - accuracy: 0.252 - 33s 63ms/step - loss: 3.2738 - accuracy: 0.252 - 33s 63ms/step - loss: 3.2733 - accuracy: 0.252 - 33

...

1875/1875 [==============================] - 126s 67ms/step - loss: 0.9065 - accuracy: 0.8151 - val_loss: 0.8770 - val_accuracy: 0.8237
Epoch 47/50
155/157 [============================>.] - ETA: 41:17 - loss: 0.7991 - accuracy: 0.812 - ETA: 13:39 - loss: 0.8932 - accuracy: 0.796 - ETA: 8:07 - loss: 0.8474 - accuracy: 0.821 - ETA: 5:45 - loss: 0.8444 - accuracy: 0.82 - ETA: 4:26 - loss: 0.8614 - accuracy: 0.81 - ETA: 3:36 - loss: 0.8684 - accuracy: 0.81 - ETA: 3:01 - loss: 0.8564 - accuracy: 0.81 - ETA: 2:35 - loss: 0.8564 - accuracy: 0.81 - ETA: 2:16 - loss: 0.8561 - accuracy: 0.81 - ETA: 2:00 - loss: 0.8513 - accuracy: 0.82 - ETA: 1:48 - loss: 0.8447 - accuracy: 0.82 - ETA: 1:37 - loss: 0.8559 - accuracy: 0.82 - ETA: 1:29 - loss: 0.8582 - accuracy: 0.82 - ETA: 1:21 - loss: 0.8630 - accuracy: 0.81 - ETA: 1:15 - loss: 0.8689 - accuracy: 0.81 - ETA: 1:09 - loss: 0.8728 - accuracy: 0.81 - ETA: 1:04 - loss: 0.8723 - accuracy: 0.81 - ETA: 1:00 - loss: 0.8661 - accuracy: 0.81 - ETA: 56s - l

1875/1875 [==============================] - 127s 68ms/step - loss: 0.8985 - accuracy: 0.8158 - val_loss: 0.9498 - val_accuracy: 0.8002
Epoch 48/50
155/157 [============================>.] - ETA: 41:18 - loss: 0.6907 - accuracy: 0.875 - ETA: 13:40 - loss: 0.8123 - accuracy: 0.864 - ETA: 8:08 - loss: 0.8738 - accuracy: 0.850 - ETA: 5:46 - loss: 0.9204 - accuracy: 0.83 - ETA: 4:26 - loss: 0.9237 - accuracy: 0.82 - ETA: 3:36 - loss: 0.9223 - accuracy: 0.82 - ETA: 3:01 - loss: 0.9225 - accuracy: 0.81 - ETA: 2:35 - loss: 0.9250 - accuracy: 0.81 - ETA: 2:16 - loss: 0.9201 - accuracy: 0.81 - ETA: 2:00 - loss: 0.9107 - accuracy: 0.81 - ETA: 1:48 - loss: 0.9047 - accuracy: 0.81 - ETA: 1:37 - loss: 0.9027 - accuracy: 0.81 - ETA: 1:29 - loss: 0.8981 - accuracy: 0.82 - ETA: 1:21 - loss: 0.9101 - accuracy: 0.81 - ETA: 1:15 - loss: 0.9019 - accuracy: 0.81 - ETA: 1:09 - loss: 0.9081 - accuracy: 0.81 - ETA: 1:04 - loss: 0.9087 - accuracy: 0.81 - ETA: 1:00 - loss: 0.9014 - accuracy: 0.81 - ETA: 56s - l

1875/1875 [==============================] - 125s 67ms/step - loss: 0.8990 - accuracy: 0.8161 - val_loss: 0.8922 - val_accuracy: 0.8174
Epoch 49/50
156/157 [============================>.] - ETA: 44:09 - loss: 0.7500 - accuracy: 0.843 - ETA: 21:59 - loss: 0.8067 - accuracy: 0.835 - ETA: 10:54 - loss: 0.9321 - accuracy: 0.808 - ETA: 7:13 - loss: 0.9241 - accuracy: 0.807 - ETA: 5:22 - loss: 0.8701 - accuracy: 0.83 - ETA: 4:15 - loss: 0.8396 - accuracy: 0.84 - ETA: 3:31 - loss: 0.8540 - accuracy: 0.84 - ETA: 2:59 - loss: 0.8600 - accuracy: 0.84 - ETA: 2:35 - loss: 0.8375 - accuracy: 0.85 - ETA: 2:16 - loss: 0.8438 - accuracy: 0.84 - ETA: 2:01 - loss: 0.8432 - accuracy: 0.84 - ETA: 1:49 - loss: 0.8419 - accuracy: 0.84 - ETA: 1:39 - loss: 0.8425 - accuracy: 0.83 - ETA: 1:30 - loss: 0.8401 - accuracy: 0.83 - ETA: 1:23 - loss: 0.8373 - accuracy: 0.83 - ETA: 1:17 - loss: 0.8344 - accuracy: 0.83 - ETA: 1:11 - loss: 0.8356 - accuracy: 0.83 - ETA: 1:08 - loss: 0.8417 - accuracy: 0.83 - ETA: 1:04 

1875/1875 [==============================] - 128s 68ms/step - loss: 0.9040 - accuracy: 0.8148 - val_loss: 0.9342 - val_accuracy: 0.8068
Epoch 50/50
156/157 [============================>.] - ETA: 41:38 - loss: 0.7185 - accuracy: 0.890 - ETA: 13:46 - loss: 0.7285 - accuracy: 0.890 - ETA: 8:12 - loss: 0.8032 - accuracy: 0.850 - ETA: 5:48 - loss: 0.7918 - accuracy: 0.85 - ETA: 4:28 - loss: 0.8128 - accuracy: 0.85 - ETA: 3:38 - loss: 0.8465 - accuracy: 0.83 - ETA: 3:03 - loss: 0.8336 - accuracy: 0.83 - ETA: 2:37 - loss: 0.8243 - accuracy: 0.83 - ETA: 2:17 - loss: 0.8277 - accuracy: 0.83 - ETA: 2:01 - loss: 0.8346 - accuracy: 0.83 - ETA: 1:49 - loss: 0.8322 - accuracy: 0.83 - ETA: 1:38 - loss: 0.8254 - accuracy: 0.83 - ETA: 1:29 - loss: 0.8246 - accuracy: 0.83 - ETA: 1:22 - loss: 0.8296 - accuracy: 0.82 - ETA: 1:15 - loss: 0.8263 - accuracy: 0.82 - ETA: 1:10 - loss: 0.8235 - accuracy: 0.83 - ETA: 1:05 - loss: 0.8279 - accuracy: 0.83 - ETA: 1:00 - loss: 0.8289 - accuracy: 0.82 - ETA: 56s - l

1875/1875 [==============================] - 126s 67ms/step - loss: 0.9031 - accuracy: 0.8152 - val_loss: 0.8964 - val_accuracy: 0.8147


In [8]:
model.evaluate(test_data)

157/157 [==============================]0.9740 - accuracy: 0.781 - 0s 76ms/step - loss: 0.9912 - accuracy: 0.773 - 0s 74ms/step - loss: 0.9246 - accuracy: 0.79 - 0s 65ms/step - loss: 0.9050 - accuracy: 0.80 - 0s 59ms/step - loss: 0.8839 - accuracy: 0.81 - 0s 56ms/step - loss: 0.8481 - accuracy: 0.82 - 0s 55ms/step - loss: 0.8476 - accuracy: 0.82 - 0s 54ms/step - loss: 0.8607 - accuracy: 0.82 - 0s 52ms/step - loss: 0.8637 - accuracy: 0.82 - 1s 53ms/step - loss: 0.8784 - accuracy: 0.81 - 1s 51ms/step - loss: 0.8549 - accuracy: 0.82 - 1s 51ms/step - loss: 0.8728 - accuracy: 0.82 - 1s 50ms/step - loss: 0.8719 - accuracy: 0.81 - 1s 48ms/step - loss: 0.8737 - accuracy: 0.81 - 1s 48ms/step - loss: 0.8774 - accuracy: 0.81 - 1s 47ms/step - loss: 0.8834 - accuracy: 0.81 - 1s 47ms/step - loss: 0.8798 - accuracy: 0.81 - 1s 47ms/step - loss: 0.8913 - accuracy: 0.81 - 1s 46ms/step - loss: 0.8930 - accuracy: 0.81 - 1s 46ms/step - loss: 0.8916 - accuracy: 0.81 - 1s 46ms/step - loss: 0.8862 - accuracy:

[0.9114261273365871, 0.8093]